In [7]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

import os
from google.colab import files

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481 (from 1)
Receiving objects: 100% (1481/1481), 53.56 MiB | 36.56 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post2-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0-py3-none-any.whl.metadata (8.3 kB)
  Using cach

  0% (0 of 1716988501) |                 | Elapsed Time: 0:00:00 ETA:  --:--:--

100% (1716988501 of 1716988501) |########| Elapsed Time: 0:00:34 Time:  0:00:34


Done.


100% (60938957 of 60938957) |############| Elapsed Time: 0:00:02 Time:  0:00:02


...

Done.


100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00
  0% (0 of 100715777) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--

Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:03 Time:  0:00:03


Done.


In [8]:
text = "i am the presdent of the usa my name is donald trump"

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

CUSTOM_VOICE_NAME = "custom"

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving 4444444.wav to 4444444.wav


In [9]:

voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

/content/tortoise-tts/tortoise/utils/audio.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sampling_rate, data = read(full_path)
/content/tortoise-tts/tortoise/models/arch_util.py:310: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Pleas

Generating autoregressive samples..


100%|██████████| 16/16 [00:46<00:00,  2.88s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 16/16 [00:18<00:00,  1.18s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [00:37<00:00, 10.68it/s]
